# Une mesure de l'inflation liée aux journaux - Projet Python pour la Data-science

### Auteurs : Lise Marchal, Raphaël Pereira et Raphaël Zambélli--Palacio

Ce notebook a pour objectif de présenter les travaux de recherche effectués dans le cadre du cours de Projet Python pour la data-science de la 2A ENSAE.

## Problématique :

Les périodes caractérisées par un taux d'inflation élevé coïcident-elles avec celles de traitement médiatique accru de cette thématique ?


In [3]:
# Imports:

import os
from dotenv import load_dotenv

import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

from scipy.stats import pearsonr, kendalltau, spearmanr, chi2_contingency, chi2
import statsmodels.api as sm



Gestion des fichiers avec SSP Cloud

In [9]:
import s3fs
import os

load_dotenv()
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

MY_BUCKET = "rapamel"
fs.ls(MY_BUCKET)
path=f"{MY_BUCKET}/diffusion/ProjetDataScienceInflation"

# 1- Statistiques descriptives des mesures d'inflation sur la période

## 1.1- Récupération et nettoyage des données quantitatives d'inflation de la FED de Saint-Louis

### 1.1.1- Utilisation de l'API de la FRED:



La Fed de Saint-Louis met à disposition via son API de nombreuses bases de données d'indicateurs macroéconomiques. Elle offre notamment des séries temporelles couvrant la quasi totalité du XXème siècle ce qui est particulièrement intéressant pour notre projet. En effet, les articles de presse après 1963 ne sont plus en libre accès. On a choisi de baser nos mesures d'inflation sur une série assez générale: "Consumer Price Index for All Urban Consumers: All Items in U.S. City Average".

In [2]:
# Création de l'URL permettant d'accéder à la série choisie

load_dotenv()
api_root="https://api.stlouisfed.org/fred/series/observations"
series_id="CPIAUCNS"
api_key=os.getenv("FREDToken") # Récupération de la clé d'API dans le fichier .env
file_type="json"

url_api = (
    f"{api_root}?"
    + f"series_id={series_id}&"
    + f"api_key={api_key}&"
    + f"file_type={file_type}"
)

In [3]:
# Lancement de la requête avec l'url créé

req = requests.get(url_api)
wb = req.json()

### 1.1.2- Nettoyage des données

#### Création et première exploration du dataframe

In [4]:
# Création d'un dataframe à partir du fichier json

CPI_Urban=pd.json_normalize(wb["observations"]) 

In [5]:
CPI_Urban.head()

,realtime_start,realtime_end,date,value
0,2024-12-12,2024-12-12,1913-01-01,9.800
1,2024-12-12,2024-12-12,1913-02-01,9.800
2,2024-12-12,2024-12-12,1913-03-01,9.800
3,2024-12-12,2024-12-12,1913-04-01,9.800
4,2024-12-12,2024-12-12,1913-05-01,9.700


In [7]:
# Sélection des colonnes utiles: date et value
CPI_Urban = CPI_Urban[["date", "value"]]


In [8]:
# Infos générales (1)
CPI_Urban.axes

[RangeIndex(start=0, stop=1343, step=1),
 Index(['date', 'value'], dtype='object')]

In [9]:
# Infos générales (2)
CPI_Urban.dtypes

date     object
value    object
dtype: object

#### Traitement des indices d'inflation

In [176]:
# Conversion en types numériques
CPI_Urban['value'] = pd.to_numeric(CPI_Urban['value'], errors='coerce')


#### Traitement des dates

In [12]:
CPI_Urban['date'] = pd.to_datetime(CPI_Urban['date'])
CPI_Urban['year'] = CPI_Urban['date'].dt.year
CPI_Urban['month'] = CPI_Urban['date'].dt.month

CPI_Urban.head()

,date,value,year,month
0,1913-01-01,9.8,1913,1
1,1913-02-01,9.8,1913,2
2,1913-03-01,9.8,1913,3
3,1913-04-01,9.8,1913,4
4,1913-05-01,9.7,1913,5


#### Création de la variable pourcentage d'inflation annuelle: 

Le jeu de données se présente sous la forme d'indices avec une base 100 en 1983. Le choix a cependant été fait de réaliser l'analyse grâce à des pourcentages d'inflation. On crée donc la variable "inf_an", qui correspond au pourcentage d'inflation calculé par rapport au niveau des prix un an plus tôt. 
\begin{equation}
    \frac{indice_0 - indice_{-12}}{indice_{-12}}*100
\end{equation}

On relèvera cependant une limite: Il n'est pas idéal d'avoir la date de la base 100 postérieure à la période étudiée.  

In [13]:
# Inflation annuelle: ((indice/indice 12 mois auparavant)-1)*100
CPI_Urban['inf_an']=((CPI_Urban['value']-CPI_Urban['value'].shift(12))/CPI_Urban['value'].shift(12))*100
CPI_Urban.head(20)

,date,value,year,month,inf_an
0,1913-01-01,9.8,1913,1,NaN
1,1913-02-01,9.8,1913,2,NaN
2,1913-03-01,9.8,1913,3,NaN
3,1913-04-01,9.8,1913,4,NaN
4,1913-05-01,9.7,1913,5,NaN
5,1913-06-01,9.8,1913,6,NaN
6,1913-07-01,9.9,1913,7,NaN
7,1913-08-01,9.9,1913,8,NaN
8,1913-09-01,10.0,1913,9,NaN
9,1913-10-01,10.0,1913,10,NaN


#### Création d'un sous-tableau de la période considérée

In [14]:
CPI_sub = CPI_Urban[(CPI_Urban['year'] > 1913) & (CPI_Urban['year'] < 1963)]
CPI_sub.head()


,date,value,year,month,inf_an
12,1914-01-01,10.0,1914,1,2.040816
13,1914-02-01,9.9,1914,2,1.020408
14,1914-03-01,9.9,1914,3,1.020408
15,1914-04-01,9.8,1914,4,0.000000
16,1914-05-01,9.9,1914,5,2.061856


In [15]:
CPI_sub.tail()

,date,value,year,month,inf_an
595,1962-08-01,30.3,1962,8,1.337793
596,1962-09-01,30.4,1962,9,1.333333
597,1962-10-01,30.4,1962,10,1.333333
598,1962-11-01,30.4,1962,11,1.333333
599,1962-12-01,30.4,1962,12,1.333333


## 1.2- Traitement des données sur l'inflation

### 1.2.1- Premières statistiques descriptives sur l'inflation pendant la période

#### Premières statistiques sur l'ensemble du dataframe (1914-1962)

In [16]:
CPI_sub['inf_an'].describe()

count    588.000000
mean       2.517532
std        6.607607
min      -15.789474
25%       -0.578035
50%        1.449275
75%        3.764927
max       23.668639
Name: inf_an, dtype: float64

Grâce à ces premières statistiques, on peut déjà constater que l'inflation annuelle moyenne entre 1914 et 1962 est positive. Elle est de plus relativement proche des cibles d'inflation traditionnelles (environ 2.5%). Cependant, on observe également une volatilité non négligeable avec un écart-type à 6.6 et surtout un pourcentage d'inflation minimum à -15.8% et un maximum à 23.7%, des niveaux respectivement bien inférieurs et supérieurs à ceux observés dans les décennies plus récentes. Le pourcentage d'inflation annuel médian est inférieur au pourcentage d'inflation moyenne ce qui indique une distribution plus dispersée vers le haut. 

#### Figure 1: Evolution du pourcentage d'inflation annuelle par mois sur l'ensemble de la période

In [29]:
# Création d'une fonction de base pour la création de graphiques en courbe

def line_graph (df, abs, ord, titre, x_title, x_format, y_title, val_rem):
    """
    Fonction traçant un graphique en courbe interactif.
    ==================================================
    Paramètres:
    ==================================================
    df: dataframe d'où sont issues les données
    abs: série sur l'axe des abcisses
    ord: série sur l'axe des ordonnées
    titre: titre du graphique
    x_title: nom de l'axe des abcisses
    y_title: nom de l'axe des ordonnées
    val_rem: booléen indiquant si l'on souhaite ou non l'affichage des valeurs remarquables (0, moyenne, médiane)
    """

    # Création du graphique
    graph = px.line(
        df, 
        x=abs, 
        y=ord, 
        title=titre,
        labels={abs: x_title, ord:y_title}
    )
    # Légende et format des axes
    graph.update_xaxes(title=x_title, tickformat=x_format)
    graph.update_yaxes(title=y_title)

    # Style du graphique
    graph.update_layout(template="plotly_dark")

    # Mise en évidence des valeurs remarquables (moyenne et médiane)
    if val_rem==True:
        mean=df[ord].mean()
        median=df[ord].median()
        
        graph.add_hline(y=mean, line_dash="dot", line_color="yellow", annotation_text="Moyenne", annotation_position="top left", annotation_font_color="yellow")
        graph.add_hline(y=median, line_dash="dot", line_color="white", annotation_text="Médiane", annotation_position="bottom right", annotation_font_color="white")
        graph.add_hline(y=0, line_color="red")

    graph.show(config={"scrollZoom": True, "displayModeBar": True})

    
    

In [30]:
line_graph (
    CPI_sub, 
    "date", 
    "inf_an", 
    "Figure 1: Pourcentage d'inflation annuelle de 1914 à 1962", 
    "Date", 
    "%m-%Y", 
    "Inflation (%)",
    True
)

On observe une **première période de très forte inflation** pendant la deuxième partie de la Première Guerre mondiale et dans l'immédiat après-guerre. L'inflation est en effet comprise entre 13% et 23,7%, le pourcentage maximum sur l'ensemble de la période considérée, atteint en juin 1920. Cette période est suivie de la **première période de déflation**, de janvier 1921 à février 1923. La déflation la plus forte de la période a lieu en juin 1921 (-15.8%). Après une relative stabilité dans les années 20, la crise des années 30 engendre une **deuxième période de déflation**, de mai 1930 à novembre 1933, l'inflation restant autour de -9.5% pendant deux ans. L'inflation reste ensuite faible jusqu'à la Seconde Guerre mondiale. Celle-ci se caractérise par une **deuxième période d'inflation**, avec un pic à 13.2% en mai 1942. Cependant, cette période est assez courte, la moyenne d'inflation étant de nouveau atteinte dès le début de 1944. L'après-guerre constitue en revanche une **troisième période de forte inflation**, avec un pic à 19.7%. La fin de la période, notamment à partir de 1952, se caractérise elle par un **stabilisation de l'inflation à des niveaux faibles**, autour de la médiane à 1.4%. 

### 1.2.2- Identification de périodes d'intérêt

#### Identification des périodes d'inflation négative

In [21]:
# Création d'une indicatrice inf_neg qui prend la valeur 1 si l'inflation est négative et 0 sinon
CPI_sub.loc[:,'inf_neg']=0
CPI_sub.loc[CPI_sub['inf_an'] < 0, 'inf_neg'] = 1
CPI_sub.head(30)

C:\Users\lisem\AppData\Local\Temp\ipykernel_41528\1604323234.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,value,year,month,inf_an,inf_neg
12,1914-01-01,10.0,1914,1,2.040816,0
13,1914-02-01,9.9,1914,2,1.020408,0
14,1914-03-01,9.9,1914,3,1.020408,0
15,1914-04-01,9.8,1914,4,0.000000,0
16,1914-05-01,9.9,1914,5,2.061856,0
17,1914-06-01,9.9,1914,6,1.020408,0
18,1914-07-01,10.0,1914,7,1.010101,0
19,1914-08-01,10.2,1914,8,3.030303,0
20,1914-09-01,10.2,1914,9,2.000000,0
21,1914-10-01,10.1,1914,10,1.000000,0


In [25]:
CPI_sub[CPI_sub['inf_neg'] == 1]['inf_an'].describe()

count    158.000000
mean      -4.149776
std        4.029451
min      -15.789474
25%       -7.680995
50%       -2.228430
75%       -0.980392
max       -0.371747
Name: inf_an, dtype: float64

On a donc 158 mois de déflation sur la période. La moyenne est assez basse à -4.1%, ce qui est plus faible que la médiane. 

In [27]:
graph=px.bar(
            CPI_sub, 
            x='date', 
            y='inf_neg', 
            title="Figure 2: Représentation des périodes d'inflation négative",
            labels={'date': 'Date', 'inf_neg':'Inflation négative'}
        )

graph.update_layout(template="plotly_dark")

graph.show(config={"scrollZoom": True, "displayModeBar": True})

#### Identification des périodes d'accélération et de descélération

Pour mesurer l'accélération et la descélération de l'inflation, le choix a été fait de s'intéresser à la différence relative par rapport au pourcentage d'inflation de l'année précédente au même mois.
En raison du nombre important de valeurs négatives, l'échelle logarithmique n'a pas été retenue malgré les avantages qu'elle aurait eu, notamment en raison des forts pourcentages d'évolution. Le choix de calculer des taux de croissance pose néanmoins le problème des valeurs nulles créant des pourcentages d'évolution infinis. On les remplace par les moyennes des valeurs précédentes et suivantes pour lisser la représentation lorsqu'elles sont isolées, sinon par les valeurs précédentes ou suivantes. 

In [31]:
# Création du pourcentage d'évolution de l'inflation
CPI_sub.loc[:, 'acceleration_12'] = (CPI_sub.loc[:,'inf_an']/CPI_sub.loc[:,'inf_an'].shift(12) - 1)*100 
CPI_sub.head(30)


C:\Users\lisem\AppData\Local\Temp\ipykernel_41528\750671830.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,value,year,month,inf_an,inf_neg,acceleration_12
12,1914-01-01,10.0,1914,1,2.040816,0,NaN
13,1914-02-01,9.9,1914,2,1.020408,0,NaN
14,1914-03-01,9.9,1914,3,1.020408,0,NaN
15,1914-04-01,9.8,1914,4,0.000000,0,NaN
16,1914-05-01,9.9,1914,5,2.061856,0,NaN
17,1914-06-01,9.9,1914,6,1.020408,0,NaN
18,1914-07-01,10.0,1914,7,1.010101,0,NaN
19,1914-08-01,10.2,1914,8,3.030303,0,NaN
20,1914-09-01,10.2,1914,9,2.000000,0,NaN
21,1914-10-01,10.1,1914,10,1.000000,0,NaN


In [32]:
# Traitement des pourcentages d'évolution infinis

# On le fait un nb arbitraire de fois pour traiter les cas de 3,4,5,6 valeurs infinies se suivant. 
# On ne fait pas une boucle while par exemple sur la moyenne pour éviter les boucles infinies

for i in range(5):
    # traitement des valeurs infinies isolées: remplacement par la moyenne entre la valeur précédente et la valeur suivante
    CPI_sub.loc[
        abs(CPI_sub["acceleration_12"]) == np.inf,
        "acceleration_12"
    ] = (CPI_sub["acceleration_12"].shift(1) + CPI_sub["acceleration_12"].shift(-1)) / 2
    #traitement des valeurs infinies suivies de valeurs infinies: remplacement par la valeur précédente
    CPI_sub.loc[
        (abs(CPI_sub["acceleration_12"]) == np.inf) & (abs(CPI_sub["acceleration_12"].shift(-1)) == np.inf),
        "acceleration_12"
    ] = CPI_sub["acceleration_12"].shift(1)

    #traitement des valeurs infinies précédées de valeurs infinies: remplacement par la valeur suivante
    CPI_sub.loc[
        (abs(CPI_sub["acceleration_12"]) == np.inf) & (abs(CPI_sub["acceleration_12"].shift(1)) == np.inf),
        "acceleration_12"
    ] = CPI_sub["acceleration_12"].shift(-1)

CPI_sub.head(20)



,date,value,year,month,inf_an,inf_neg,acceleration_12
12,1914-01-01,10.0,1914,1,2.040816,0,NaN
13,1914-02-01,9.9,1914,2,1.020408,0,NaN
14,1914-03-01,9.9,1914,3,1.020408,0,NaN
15,1914-04-01,9.8,1914,4,0.000000,0,NaN
16,1914-05-01,9.9,1914,5,2.061856,0,NaN
17,1914-06-01,9.9,1914,6,1.020408,0,NaN
18,1914-07-01,10.0,1914,7,1.010101,0,NaN
19,1914-08-01,10.2,1914,8,3.030303,0,NaN
20,1914-09-01,10.2,1914,9,2.000000,0,NaN
21,1914-10-01,10.1,1914,10,1.000000,0,NaN


In [33]:
line_graph (
    CPI_sub, 
    "date", 
    "acceleration_12", 
    "Figure 3: Pourcentage d'accélération de l'inflation annuelle par rapport à sa valeur l'année précédente de 1915 à 1962", 
    "Date", 
    "%Y", 
    "Accélération (%)",
    False
)

On constate que les périodes d'accélération et de déscélération sont souvent les mêmes. On observe cinq pics de volatilité: autour de 1916, de 1931, de 1941, de 1947 et de 1951. Ils correspondent donc tous à des périodes de forte inflation ou déflation.

# 2- Analyse de fréquence sur le corpus de presse

## 2.1- Récupération et formatage des dataframes de fréquence

**Note**: En raison du temps d'exécution important nécessaire à la réalisation des fichiers csv. de fréquence, ceux-ci sont générés à part et seulement importés ici.

AJOUTER TEXTE SUR CHOIX ET BDD

### 2.1.1- Création de tables par période et d'une table sur toute la période

In [34]:
# AJOUTER ANNEES 20
# Récupération des données dans un dictionnaire par décennie
dates=["1930_1939","1940_1949","1950_1959","1960_1963"]

df = {}

for date in dates: 
    url=(
        "frequences_data_"
        + f"{date}"
        +".csv"
    )
    
    df[f"freq_{date}"] = pd.read_csv(url)




In [35]:
# AJOUTER ANNEES 20
# Création d'une table sur toute la période AJOUTER ANNEES 20
freq_tot=pd.concat([df["freq_1930_1939"],df["freq_1940_1949"],df["freq_1950_1959"],df["freq_1960_1963"]])

In [36]:
# Affichage de toutes les colonnes et de toutes les lignes
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

### 2.1.1- Premières informations sur les données

In [37]:
freq_tot.head()

,key,nbre_articles,inflation,disinflation,inflationary,deflation,prices,cost,wages,currency,money,devaluation,recession,stagflation,economy,market,increase,decrease,cpi,price level,wage growth,economic downturn,monetary policy,cost increase,cost reduction,market prices,inflation rate,interest rates,price stability,consumption basket,purchasing power,consumer price index,rise in prices,fall in prices,cost of living,inflation expectations,money supply growth,central bank policy,economic price adjustments
0,1930-01,37954,0,0,0,2,529,1359,96,27,1240,0,11,0,77,1044,987,159,24,11,0,0,0,0,0,15,0,12,0,0,8,0,0,0,11,0,0,0,0
1,1930-02,35029,0,0,0,3,429,1217,91,40,1170,0,13,0,85,831,711,92,38,5,0,0,0,0,0,11,0,3,0,0,7,0,0,1,7,0,0,0,0
2,1930-03,41033,0,0,0,1,551,1443,137,30,1513,0,29,0,93,991,826,125,16,13,0,0,0,1,0,13,0,15,0,0,3,0,0,0,7,0,0,0,0
3,1930-04,38407,1,0,0,3,520,1419,99,28,1304,0,24,0,104,907,888,132,12,8,0,0,0,0,0,12,0,4,1,0,9,0,2,0,4,0,0,0,0
4,1930-05,39827,0,0,0,3,634,1358,94,30,1189,0,23,0,94,1032,917,170,16,12,0,0,0,0,0,6,0,7,0,0,5,0,0,0,13,0,0,0,0


In [39]:
freq_tot.dtypes

key                           object
nbre_articles                  int64
inflation                      int64
disinflation                   int64
inflationary                   int64
deflation                      int64
prices                         int64
cost                           int64
wages                          int64
currency                       int64
money                          int64
devaluation                    int64
recession                      int64
stagflation                    int64
economy                        int64
market                         int64
increase                       int64
decrease                       int64
cpi                            int64
price level                    int64
wage growth                    int64
economic downturn              int64
monetary policy                int64
cost increase                  int64
cost reduction                 int64
market prices                  int64
inflation rate                 int64
i

In [40]:
# Formatage des dates

freq_tot = freq_tot.rename(columns={'key': 'date'}) # Renommage de la colonne 'key' en 'date'
freq_tot['date'] = pd.to_datetime(freq_tot['date'])
freq_tot['year'] = freq_tot['date'].dt.year
freq_tot['month'] = freq_tot['date'].dt.month

## 2.2- Sélection des termes du champ lexical de l'inflation et des prix les plus utilisés dans les articles: 

On cherche à sélectionner les termes les plus utilisés pour permettre de produire une analyse plus pertinente et lisible.

In [42]:
# Listes utiles (1)

# Liste comprenant les colonnes autres que les termes du champ lexical de l'inflation
autres = ['date','year', 'month', 'nbre_articles']

In [43]:
# Affichage des statistiques descriptives pour les termes du champ lexical et classement par ordre décroissant des moyennes d'occurence
desc=freq_tot.drop(columns=autres).describe()
desc_transposed = desc.transpose()
desc_sorted = desc_transposed.sort_values(by="mean", ascending=False)
desc_sorted

,count,mean,std,min,25%,50%,75%,max
cost,408.0,1505.284314,630.851819,141.0,1273.25,1569.5,1924.25,2841.0
money,408.0,1186.750000,622.774671,109.0,863.00,1130.0,1497.50,2892.0
increase,408.0,927.394608,448.287770,66.0,640.75,929.5,1255.00,1975.0
market,408.0,749.002451,403.730038,68.0,471.50,719.0,1026.25,2007.0
prices,408.0,724.061275,373.814817,46.0,432.75,796.5,985.25,1919.0
wages,408.0,201.414216,145.236966,7.0,80.25,195.5,284.00,895.0
economy,408.0,185.299020,101.788811,18.0,131.00,176.0,223.25,715.0
decrease,408.0,105.144608,86.065749,3.0,40.00,74.5,161.25,470.0
currency,408.0,69.642157,83.742611,0.0,25.00,44.0,78.25,696.0
recession,408.0,25.387255,37.380007,0.0,8.00,17.5,30.00,427.0


On remarque que certains termes sont beaucoup plus utilisés que d'autres: *cost, money, increase, market, prices*, etc.
Pour certains comme increase, cela peut sûrement être dû à leur utilisation dans des contextes très différents. Pour d'autres, comme price or cost, on peut considérer que cela donne une indication du traitement de thématiques liées à l'inflation. On peut envisager que ce thème soit plus traité par l'évocation des prix ou des coûts que par des termes plus "technique" comme *deflation, inflationary* ou encore *money supply growth*.
Enfin, ceci peut aussi être lié à la base de données utilisée, même si le nombre important d'articles peut laisser supposer une certaine représentativité. 

In [44]:
# Listes utiles (2):

# Liste des termes les plus utilisés dans les articles
selected=desc_sorted.index[desc_sorted['mean']>11].to_list()

# Liste des colonnes du dataframe réduit
sub=autres+selected

# Liste des variables numériques
num=selected+['nbre_articles']



In [46]:
# Création d'un dataframe réduit
freq_sub=freq_tot[sub]
freq_sub.head()

,date,year,month,nbre_articles,cost,money,increase,market,prices,wages,economy,decrease,currency,recession,cost of living,purchasing power,inflation,price level
0,1930-01-01,1930,1,37954,1359,1240,987,1044,529,96,77,159,27,11,11,8,0,11
1,1930-02-01,1930,2,35029,1217,1170,711,831,429,91,85,92,40,13,7,7,0,5
2,1930-03-01,1930,3,41033,1443,1513,826,991,551,137,93,125,30,29,7,3,0,13
3,1930-04-01,1930,4,38407,1419,1304,888,907,520,99,104,132,28,24,4,9,1,8
4,1930-05-01,1930,5,39827,1358,1189,917,1032,634,94,94,170,30,23,13,5,0,12


## 2.3- Premières visualisations des fréquences

In [47]:
# Création d'une fonction de base diagramme à barres empilées

def stacked_bar(df,abs,sections,titre,x_title,y_title, var_title):
    """
    Fonction traçant un diagramme à barres empilées.
    ==================================================
    Paramètres:
    ==================================================
    df: dataframe d'où sont issues les données
    abs: série sur l'axe des abcisses
    ord: sections découpant chaque barre
    titre: titre du graphique
    x_title: nom de l'axe des abcisses
    y_title: nom de l'axe des ordonnées
    var_title: nom des sections des barres
    """
    graph=px.bar(
            df, 
            x=abs, 
            y=sections, 
            title=titre,
            labels={abs: x_title, 'value':y_title, 'variable':var_title}
        )

    graph.update_layout(template="plotly_dark")

    graph.show(config={"scrollZoom": True, "displayModeBar": True})


### 2.3.1- Fréquence d'apparition des termes par date

In [48]:
# Création d'un graphique représentant le nombre d'articles mentionnant les termes sélectionnés par année:

stacked_bar(
    freq_sub,
    'date',
    selected,
    "Figure 4: Occurence des termes les plus utilisés dans le champ lexical de l'inflation de 1914 à 1962",
    "Date",
    "Fréquence",
    "Termes du champ lexical de l'inflation"
)

On constate bien la prévalence des termes *cost*, *money*, *increase*, *market* et *price*. On note également les très faibles occurences dans la deuxième moitié des années 50. Ceci s'explique par le nombre d'articles plus limité présent dans la base pour cette période, sans qu'aucune explication ne soit fournie. On veillera donc à ne pas surinterpréter d'éventuels résultats sur cette période. On peut également s'intéresser à une analyse en termes de proportion pour éviter une influence trop forte de la variable nombre d'articles.

### 2.3.2- Pourcentage d'articles mentionnant les différents termes 

In [50]:
# Créations des pourcentages

for col in selected:
    freq_sub.loc[:,f"{col}_rat"]=(freq_sub.loc[:,f"{col}"]/freq_sub.loc[:,'nbre_articles'])*100

freq_sub.head(2)

,date,year,month,nbre_articles,cost,money,increase,market,prices,wages,economy,decrease,currency,recession,cost of living,purchasing power,inflation,price level,cost_rat,money_rat,increase_rat,market_rat,prices_rat,wages_rat,economy_rat,decrease_rat,currency_rat,recession_rat,cost of living_rat,purchasing power_rat,inflation_rat,price level_rat
0,1930-01-01,1930,1,37954,1359,1240,987,1044,529,96,77,159,27,11,11,8,0,11,3.580650,3.267113,2.600516,2.750698,1.393792,0.252938,0.202877,0.418928,0.071139,0.028982,0.028982,0.021078,0.0,0.028982
1,1930-02-01,1930,2,35029,1217,1170,711,831,429,91,85,92,40,13,7,7,0,5,3.474264,3.340090,2.029747,2.372320,1.224700,0.259785,0.242656,0.262640,0.114191,0.037112,0.019983,0.019983,0.0,0.014274


In [51]:
# Listes utiles (3): 
# Pourcentage des différents termes du champ lexical de l'inflation
selected_rat=[col for col in freq_sub.columns if col not in sub]


In [52]:
stacked_bar(
    freq_sub,
    'date',
    selected_rat,
    "Figure 5: Pourcentage d'articles contenant les termes les plus utilisés dans le champ lexical de l'inflation de 1914 à 1962",
    "Date",
    "Pourcentage",
    "Terme"
)

On peut tout d'abord s'intéresser au niveau agrégé. On constate un pic d'articles mentionnant des termes liés à l'inflation autour de 1933, avec plus de 20% d'entre-eux. Cela coïncide avec la crise économique des années 30, période de forte déflation. La proportion d'article traitant de ces sujets baisse ensuite, notamment pendant la guerre. On peut envisager que d'autres sujets étaient alors jugés plus importants, même si cela semble rester un sujet, les pourcentages étant toujours relativement importants. On constate un deuxième pic, de même niveau que celui des années 30, à la fin des années 40, là encore période de forte inflation. La proportion diminue encore notamment à la fin des années 50. Cela pourrait correspondre à la stabilisation de l'inflation mais comme mentionné ci-dessus, moins d'articles sont disponibles sur cette période donc cela pourrait également jouer.

### 2.3.3- Fréquence annuelle d'apparition des termes

Pour plus de lisibilité, on peut également s'intéresser aux fréquences par année et non plus par mois. On a cependant souhaité commencer par les fréquences mensuelles pour conserver un maximum d'information.

In [53]:
# Création d'une table annuelle: on somme les variables numériques par année

freq_an = freq_sub.groupby(['year'])[num].sum()

freq_an=freq_an.reset_index() # pour que year soit une colonne à part entière

freq_an.head(20)

,year,cost,money,increase,market,prices,wages,economy,decrease,currency,recession,cost of living,purchasing power,inflation,price level,nbre_articles
0,1930,17230,17390,11349,12681,8488,1340,1222,2135,460,393,129,186,4,199,492695
1,1931,17147,18599,12295,11870,9456,2129,1721,2545,735,326,147,299,25,268,493816
2,1932,28882,27850,16642,17492,13901,2753,5386,3830,2186,372,180,563,71,380,664615
3,1933,29775,28858,18679,19055,17304,5123,5354,3392,4888,463,412,1337,547,804,642380
4,1934,28125,27342,18342,17653,13878,3924,2718,2731,2642,437,331,694,279,387,654342
5,1935,25665,24510,16167,13657,12275,3000,2107,2344,1940,376,258,425,238,230,635554
6,1936,24984,23518,16374,13836,11706,3060,2092,2232,1760,314,260,401,180,176,662015
7,1937,26167,21141,17382,13604,12423,4576,2519,2127,1172,967,353,405,80,191,676549
8,1938,25482,22922,15082,13641,11297,3485,2471,2489,1239,1681,192,458,84,173,665274
9,1939,21572,17466,14268,11620,10742,2228,3071,1987,1129,395,141,268,42,143,556283


In [54]:
# Graphique des fréquences annuelles

stacked_bar(
    freq_an,
    'year',
    selected,
    "Figure 6: Fréquence annuelle des termes les plus utilisés dans le champ lexical de l'inflation de 1914 à 1962",
    "Année",
    "Fréquence",
    "Terme"
)

### 2.3.4- Pourcentages annuels d'apparition des termes

In [55]:
# Création des pourcentages
for col in selected:
    freq_an.loc[:,f"{col}_rat"]=(freq_an.loc[:,f"{col}"]/freq_an.loc[:,'nbre_articles'])*100

freq_an.head(2)

,year,cost,money,increase,market,prices,wages,economy,decrease,currency,recession,cost of living,purchasing power,inflation,price level,nbre_articles,cost_rat,money_rat,increase_rat,market_rat,prices_rat,wages_rat,economy_rat,decrease_rat,currency_rat,recession_rat,cost of living_rat,purchasing power_rat,inflation_rat,price level_rat
0,1930,17230,17390,11349,12681,8488,1340,1222,2135,460,393,129,186,4,199,492695,3.497093,3.529567,2.303453,2.573803,1.722770,0.271974,0.248024,0.433331,0.093364,0.079765,0.026183,0.037752,0.000812,0.040390
1,1931,17147,18599,12295,11870,9456,2129,1721,2545,735,326,147,299,25,268,493816,3.472346,3.766383,2.489794,2.403729,1.914883,0.431132,0.348510,0.515374,0.148841,0.066016,0.029768,0.060549,0.005063,0.054271


In [57]:
# Graphique des pourcentages d'apparition annuels
stacked_bar(
    freq_an,
    'year',
    selected_rat,
    "Figure 7: Pourcentage d'apparition annuelle des termes les plus utilisés dans le champ lexical de l'inflation de 1914 à 1962",
    "Année",
    "Pourcentage",
    "Terme"
)

On constate ici aussi clairement deux pics, en 1933 et en 1947. La proportion des différents termes reste relativement constante.

# 3- Comparaisons des statistiques d'inflation et des statistiques de fréquences

## 3.1- Analyse agrégée

### 3.1.1- Analyse d'éventuelles corrélations entre l'inflation annuelle et le pourcentage total d'articles mentionnant des termes liés à l'inflation

#### Création du pourcentage total d'articles mentionnant les termes du champ lexical de l'inflation

In [59]:
# Analyse agrégée: (somme du nombre d'occurence pour chaque terme/ nbre d'articles)*100
freq_sub.loc[:,'total_rat'] = (freq_sub.loc[:,selected].sum(axis=1)/freq_sub.loc[:,"nbre_articles"])*100
freq_sub.head(1)

,date,year,month,nbre_articles,cost,money,increase,market,prices,wages,economy,decrease,currency,recession,cost of living,purchasing power,inflation,price level,cost_rat,money_rat,increase_rat,market_rat,prices_rat,wages_rat,economy_rat,decrease_rat,currency_rat,recession_rat,cost of living_rat,purchasing power_rat,inflation_rat,price level_rat,total_rat
0,1930-01-01,1930,1,37954,1359,1240,987,1044,529,96,77,159,27,11,11,8,0,11,3.58065,3.267113,2.600516,2.750698,1.393792,0.252938,0.202877,0.418928,0.071139,0.028982,0.028982,0.021078,0.0,0.028982,14.646678


#### Visualisation

In [60]:
def two_graphs(df1_x,df1_y,df2_x,df2_y,df1_name,df2_name, abs_name, titre):
    """
    Fonction traçant deux courbes sur un même graphique
    pour comparer les évolutions.
    ==================================================
    Paramètres:
    ==================================================
    df1_x: abscisse de la première courbe
    df1_y: ordonnée de la première courbe
    df2_x: abscisse de la deuxième courbe
    df2_y: ordonnée de la deuxième courbe
    df1_name: nom de la première courbe
    df2_name: nom de la deuxième courbe
    abs_name: nom de l'axe des abcisses
    titre: titre du graphique
    """

    graph = go.Figure()

    graph.add_trace(go.Scatter(
        x=df1_x, 
        y=df1_y, 
        mode='lines', 
        name=df1_name,
        yaxis='y' 
    ))

    graph.add_trace(go.Scatter(
        x=df2_x, 
        y=df2_y, 
        mode='lines', 
        name=df2_name,
        yaxis='y2'  
    ))

    graph.update_layout(
        title=titre,
        xaxis=dict(title=abs_name),
        yaxis=dict(title=df1_name, side='left'),
        yaxis2=dict(
            title=df2_name,
            side='right',
            overlaying='y'  
        ),
        template="plotly_dark"
)


    graph.show(config={"scrollZoom": True, "displayModeBar": True})

In [64]:
# AJOUTER BONNE DATE
CPI_40=CPI_sub[CPI_sub['year']>1929] !

two_graphs(
    CPI_40['date'],
    abs(CPI_40['inf_an']),
    freq_sub['date'],
    freq_sub['total_rat'],
    'Inflation (%, val abs.)',
    "Articles (%)", 
    "Date", 
    "Figure 8: Comparaison du pourcentage d'inflation annuelle (en valeur absolue) et du pourcentage d'articles contenant des termes liés"
    )

On a choisi de comparer le pourcentage d'articles mentionnant des termes liés à l'inflation avec la valeur absolue de l'inflation pour prendre en compte l'ampleur des évolutions de prix et donc potentiellement du traitement de la thématique et pas leur direction.

De prime abord, il semble y avoir une corrélation entre ces deux variables: c'est ce qu'on va tester dans les sections suivantes. 

#### Tests de corrélation entre la valeur absolue de l'inflation annuelle et le pourcentage d'articles mentionnant des termes liés

Le choix a été fait de réaliser différents tests de corrélation entre l'inflation et le pourcentage d'articles mentionnant des termes liés. On garde une mesure mensuelle pour conserver un maximum d'information et on prend des pourcentages pour ne pas fausser les tests par un nombre d'articles différent.
- test de corrélation paramétrique: Pearson, test de corrélation linéaire. 
\begin{equation}
    Corr(X,Y)=\frac{Cov(X,Y)}{\sqrt{Var(X)*Var(Y)}}
\end{equation}
- tests de corrélation non paramétrique (basés sur le rang): 
-> tau de Kendall  
\begin{equation}
    \tau = \frac{C - D}{\frac{1}{2}n(n-1)}
\end{equation}
avec C le nombre de paires concordantes (qui conservent le rang) D le nombre de paires discordantes et n le nombre d'observations. 

-> rho de Spearman
\begin{equation}
    \rho = 1 - \frac{6 \sum d_i^2}{n(n^2 - 1)}
\end{equation}
avec d la différence de rang pour chaque observation et n le nombre total d'observations

Posent l'hypothèse nulle que les deux variables sont indépendantes. On pose un seuil de significativité à 5%.


In [67]:
# Fusion des tables de fréquence et de pourcentage d'inflation CHANGER CPI_40
inf_freq=pd.merge(CPI_40, freq_sub, on=['date', 'year', 'month'])

inf_freq.head()


,date,value,year,month,inf_an,inf_neg,acceleration_12,nbre_articles,cost,money,increase,market,prices,wages,economy,decrease,currency,recession,cost of living,purchasing power,inflation,price level,cost_rat,money_rat,increase_rat,market_rat,prices_rat,wages_rat,economy_rat,decrease_rat,currency_rat,recession_rat,cost of living_rat,purchasing power_rat,inflation_rat,price level_rat,total_rat
0,1930-01-01,17.1,1930,1,0.000000,0,-100.000000,37954,1359,1240,987,1044,529,96,77,159,27,11,11,8,0,11,3.580650,3.267113,2.600516,2.750698,1.393792,0.252938,0.202877,0.418928,0.071139,0.028982,0.028982,0.021078,0.000000,0.028982,14.646678
1,1930-02-01,17.0,1930,2,-0.584795,1,-49.705882,35029,1217,1170,711,831,429,91,85,92,40,13,7,7,0,5,3.474264,3.340090,2.029747,2.372320,1.224700,0.259785,0.242656,0.262640,0.114191,0.037112,0.019983,0.019983,0.000000,0.014274,13.411745
2,1930-03-01,16.9,1930,3,-0.588235,1,0.588235,41033,1443,1513,826,991,551,137,93,125,30,29,7,3,0,13,3.516682,3.687276,2.013014,2.415129,1.342822,0.333878,0.226647,0.304633,0.073112,0.070675,0.017059,0.007311,0.000000,0.031682,14.039919
3,1930-04-01,17.0,1930,4,0.591716,0,-150.591716,38407,1419,1304,888,907,520,99,104,132,28,24,4,9,1,8,3.694639,3.395214,2.312079,2.361549,1.353920,0.257766,0.270784,0.343687,0.072903,0.062489,0.010415,0.023433,0.002604,0.020830,14.182311
4,1930-05-01,16.9,1930,5,-0.588235,1,-49.411765,39827,1358,1189,917,1032,634,94,94,170,30,23,13,5,0,12,3.409747,2.985412,2.302458,2.591207,1.591885,0.236021,0.236021,0.426846,0.075326,0.057750,0.032641,0.012554,0.000000,0.030130,13.987998


In [144]:
def corr_tests(list1,list2):
    # Test de corrélation de Pearson
    print("TEST DE CORRELATION DE PEARSON:")
    r, p = pearsonr(list1, list2)
    print ("Le coefficient de corrélation de Pearson s'élève à: ", r, " et sa p-value est de: ", p,".")
    if p<0.05:
        print("Le coefficient est donc significatif.")
        if r<0.30:
            print("Les variables sont de plus faiblement corrélées.")
        elif r>0.70:
            print("Les variables sont de plus fortement corrélées.")
        else: 
            print("Les variables sont de plus moyennement corrélées.")
    else:
        print("Le coefficient n'est donc pas significatif.")

    # Tau de Kendall
    print("TAU DE KENDALL:")
    tau,p=kendalltau(list1, list2)
    print("Le tau de Kendall s'élève à: ", tau, " et sa p-value est de: ", p, ".")
    if p<0.05:
        print("Le coefficient est donc significatif.")
        if tau<0.30:
            print("Les variables sont de plus faiblement corrélées.")
        elif tau>0.70:
            print("Les variables sont de plus fortement corrélées.")
        else: 
            print("Les variables sont de plus moyennement corrélées.")
    else:
        print("Le coefficient n'est donc pas significatif.")

    # Rho de Spearman
    print("RHO DE SPEARMAN:")
    rho, p = spearmanr(abs(inf_freq["inf_an"]), inf_freq["total_rat"])
    print("Le coef de Spearman s'élève à: ", rho, " avec une p-value de: ", p, ".") 
    if p<0.05:
        print("Le coefficient est donc significatif.")
        if rho<0.30:
            print("Les variables sont de plus faiblement corrélées.")
        elif rho>0.70:
            print("Les variables sont de plus fortement corrélées.")
        else: 
            print("Les variables sont de plus moyennement corrélées.")
    else:
        print("Le coefficient n'est donc pas significatif.")  

In [71]:
# Tests de corrélation entre la valeur absolue de l'inflation annuelle et le pourcentage d'apparition des termes du champ lexical de l'inflation
corr_tests(abs(inf_freq['inf_an']),inf_freq['total_rat'])

TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  0.4347079009584339  et sa p-value est de:  1.0997764041821692e-19 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  0.2575009311315976  et sa p-value est de:  2.1759292708742675e-14 .
Le coefficient est donc significatif.
Les variables sont de plus faiblement corrélées.
RHO DE SPEARMAN:
Le coef de Spearmanr s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


#### 3.1.2- Analyse via les moyennes mobiles (ma)

#### Moyennes mobiles

L'utilisation de moyennes mobiles vise à supprimer les fluctuations transitoires pour mettre en valeur les tendances de long terme. On choisit ici une fenêtre d'un an.

In [72]:
# Moyennes mobiles de l'inflation annuelle et du pourcentage d'articles mentionnant des termes liés avec une fenêtre d'un an
inf_freq['inf_ma'] = inf_freq['inf_an'].rolling(window=12).mean()
inf_freq['tot_rat_ma'] = inf_freq['total_rat'].rolling(window=12).mean()

#### Visualisation

In [73]:
# Visualisation après lissage via moyennes mobiles
two_graphs(
    inf_freq['date'],
    abs(inf_freq['inf_ma']),
    inf_freq['date'],
    inf_freq['tot_rat_ma'],
    'Inflation ma (%, val abs.)',
    "Articles ma (%)", 
    "Date", 
    "Figure 9: Comparaison des moyennes mobiles de l'inflation annuelle (en valeur absolue) et du pourcentage d'articles mentionnant des termes liés"
    )

On constate bien l'effet du lissage. De plus, la corrélation semble bien se maintenir.

#### Tests de corrélation entre la valeur absolue de l'inflation annuelle en ma et le pourcentage d'articles mentionnant des termes liés en ma

In [74]:
# Corrélations entre variables lissées
inf_freq_cleaned = inf_freq.replace([np.inf, -np.inf], np.nan).dropna(subset=['inf_ma', 'tot_rat_ma'])
corr_tests(abs(inf_freq_cleaned['inf_ma']),inf_freq_cleaned['tot_rat_ma'])

TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  0.5050288165416085  et sa p-value est de:  2.602508991982939e-26 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  0.24130630361374422  et sa p-value est de:  1.5508011548218677e-12 .
Le coefficient est donc significatif.
Les variables sont de plus faiblement corrélées.
RHO DE SPEARMAN:
Le coef de Spearmanr s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


On retrouve les mêmes ordres de grandeur que dans les tests précédents, avec des variables positivement faiblement ou moyennement corrélées. Le coefficient de Pearson est de plus plus élevé.

### 3.1.3- Comparaison via les quartiles

On va maintenant classer chaque observation selon les quartiles d'inflation et de fréquence pour comparer les deux variables.

In [91]:
def quart(name,var): 
    """
    Fonction classant les observations en quatre catégories basés sur les quartiles d'une variable.
    ==============================================================================================
    Paramètres
    ==============================================================================================
    name: nom de la variable catégorielle prenant les valeurs 1,2,3 et 4: par ex "inf_qu"
    var: série dont sont calculés les quartiles: par ex: inf_freq["inf_an"]
    """
    inf_freq.loc[:,name]=0

    q1=var.quantile(0.25)
    q2=var.quantile(0.50)
    q3=var.quantile(0.75)
    
    inf_freq.loc[
        var<q1, 
        name
        ] = 1
    inf_freq.loc[
        (var>=q1) & (var<q2), 
        name
        ] = 2
    inf_freq.loc[
        (var>=q2) & (var<q3), 
        name
        ] = 3
    inf_freq.loc[
        var>=q3, 
        name
        ] = 4

In [93]:
# Classement des dates en quatre catégories selon les quartiles des pourcentages du nombre total d'articles 
quart('tot_rat_qu',inf_freq['total_rat'])


In [118]:
# Classement des dates en quatre catégories selon les quartiles des pourcentages d'inflation annuelle
quart('inf_qu', abs(inf_freq['inf_an']))

In [119]:
# Création d'un dataframe 4*4 vide
freq = ['Freq: quartile 1', 'Freq: quartile 2', 'Freq: quartile 3', 'Freq: quartile 4']
inf = ['Inflation: quartile 1', 'Inflation: quartile 2', 'Inflation: quartile 3', 'Inflation: quartile 4']


quartiles = pd.DataFrame(columns=freq, index=inf)



In [120]:
# Remplissage du dataframe par le nombre d'observations dans le quartile d'inflation i+1 et dans le quartile de fréquence j+1 pour i et j allant de 0 à 3
for i in range(4):
    for j in range(4):
        count=inf_freq[(inf_freq['inf_qu']==i+1) & (inf_freq['tot_rat_qu']==j+1)].shape[0]
        quartiles.iloc[i,j]=count


quartiles


,Freq: quartile 1,Freq: quartile 2,Freq: quartile 3,Freq: quartile 4
Inflation: quartile 1,33,28,23,14
Inflation: quartile 2,35,27,27,11
Inflation: quartile 3,23,28,27,21
Inflation: quartile 4,8,16,22,53


#### Visualisation

In [122]:
heatmap = px.imshow(
    quartiles,
    text_auto=True,
    labels=dict(x="Articles", y="Inflation", color="Nb observations"),
    title="Figure 10: Croisement des quartiles d'inflation (valeur absolue) et des quartiles de fréquence"
    )

heatmap.update_layout(template="plotly_dark")

heatmap.show(config={"scrollZoom": True, "displayModeBar": True})

#### Test du chi 2 de Pearson


\begin{equation}
    \chi^2 = \sum \frac{(O_i - E_i)^2}{E_i}
\end{equation}


où O sont les fréquences observées et E les fréquences attendues (sous l'hypothèse nulle d'indépendance).

On rejette l'hypothèse nulle si la statistique du chi2 est supérieure à uen valeur critique qui dépend des degrés de liberté et d'un seuil qu'on fixe à 0.05.

In [126]:
for col in quartiles.columns:
    col = pd.to_numeric(col, errors='coerce')

In [127]:
array=quartiles.to_numpy().tolist()
print(array)

[[33, 28, 23, 14], [35, 27, 27, 11], [23, 28, 27, 21], [8, 16, 22, 53]]


In [141]:
stat, p, dof, expected =chi2_contingency(array)
print("Statistique du chi2:", stat)
print("P-value:", p)
print("Degrés de liberté:", dof)
print("Fréquences attendues:\n", expected)
critical_value = chi2.ppf(1 - 0.05, dof)
if p<0.05 and stat>critical_value:
    print("Les deux variables sont bien associées.")


Statistique du chi2: 68.50384662956091
P-value: 2.989816202617261e-11
Degrés de liberté: 9
Fréquences attendues:
 [[24.5  24.5  24.5  24.5 ]
 [25.   25.   25.   25.  ]
 [24.75 24.75 24.75 24.75]
 [24.75 24.75 24.75 24.75]]
Les deux variables sont bien associées.


## 3.2- Prolongements de l'analyse 

### 3.2.1- Analyse du lien entre accélération de l'inflation et traitement médiatique

#### Visualisation

In [142]:
two_graphs(
    CPI_40['date'],
    abs(CPI_40['acceleration_12']),
    freq_sub['date'],
    freq_sub['total_rat'],
    'Accélération (%, val abs.)',
    "Articles (%)", 
    "Date", 
    "Figure 11: Comparaison de l'accélération (en valeur absolue) et du pourcentage d'articles mentionnant des termes liés"
    )

Il semblerait que la corrélation soit moins forte entre ces deux variables.

#### Tests de corrélation entre la valeur absolue du pourcentage d'accélération de l'inflation et le pourcentage d'articles mentionnant des termes liés à l'inflation

In [143]:
# Nouvelle corrélation:
inf_freq_cleaned = inf_freq.replace([np.inf, -np.inf], np.nan).dropna(subset=['acceleration_12', 'total_rat'])
corr_tests(abs(inf_freq_cleaned['acceleration_12']),inf_freq_cleaned['total_rat'])


TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  -0.0434309980150001  et sa p-value est de:  0.39114179050451087 .
Le coefficient n'est donc pas significatif.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  -0.027158306895365858  et sa p-value est de:  0.42227376133969396 .
Le coefficient n'est donc pas significatif.
RHO DE SPEARMAN:
Le coef de Spearmanr s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


On voit que deux des trois tests sont non significatifs. On ne peut donc pas conclure à une association entre ces deux variables.

### 3.2.2- Analyse par terme 

On peut également s'intéresser à des termes en particulier, notamment ceux qu'on a relevés comme les plus fréquents.

#### Cost

In [146]:
two_graphs(
    CPI_40['date'],
    abs(CPI_40['inf_an']),
    freq_sub['date'],
    freq_sub['cost_rat'],
    'Inflation (%, val abs.)',
    "Cost (%)", 
    "Date", 
    "Figure 12: Comparaison du pourcentage d'inflation annuelle (en valeur absolue) et du pourcentage d'apparition de cost"
    )

Là-encore, la forte variabilité du pourcentage d'articles mentionnant le terme cost ne semble pas aller dans le sens d'une corrélation.

In [147]:
corr_tests(abs(inf_freq['inf_an']),inf_freq['cost_rat'])

TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  0.052208221263109256  et sa p-value est de:  0.3000374105856925 .
Le coefficient n'est donc pas significatif.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  -0.024885566236692074  et sa p-value est de:  0.4603149007080154 .
Le coefficient n'est donc pas significatif.
RHO DE SPEARMAN:
Le coef de Spearman s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


Deux tests sur trois sont non significatifs, on ne peut donc pas conclure à une association des deux variables. On peut néanmoins tenter de les lisser.

In [148]:
# en ma
inf_freq['cost_rat_ma'] = inf_freq['cost_rat'].rolling(window=12).mean()
inf_freq_cleaned = inf_freq.replace([np.inf, -np.inf], np.nan).dropna(subset=['inf_ma', 'cost_rat_ma'])
corr_tests(abs(inf_freq_cleaned['inf_ma']),inf_freq_cleaned['cost_rat_ma'])


TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  0.017996942989556074  et sa p-value est de:  0.7248339096562906 .
Le coefficient n'est donc pas significatif.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  -0.08575661033848318  et sa p-value est de:  0.011987078945064433 .
Le coefficient est donc significatif.
Les variables sont de plus faiblement corrélées.
RHO DE SPEARMAN:
Le coef de Spearman s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


Un test supplémentaire devient significatif et met en évidence une faible corrélation. On en conclut donc à peu voire pas du tout d'association entre les variables.

#### Money

In [151]:
two_graphs(
    CPI_40['date'],
    abs(CPI_40['inf_an']),
    freq_sub['date'],
    freq_sub['money_rat'],
    'Inflation (%, val abs.)',
    "Money (%)", 
    "Date", 
    "Figure 13: Comparaison du pourcentage d'inflation annuelle (en valeur absolue) et du pourcentage d'apparition de money"
    )

Au vu de la forte volatilité, on vérifie tout en suite en moyennes mobiles.

In [150]:
# en ma
inf_freq['money_rat_ma'] = inf_freq['money_rat'].rolling(window=12).mean()
inf_freq_cleaned = inf_freq.replace([np.inf, -np.inf], np.nan).dropna(subset=['inf_ma', 'money_rat_ma'])
corr_tests(abs(inf_freq_cleaned['inf_ma']),inf_freq_cleaned['money_rat_ma'])

TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  0.1880995270229654  et sa p-value est de:  0.00020565682364216044 .
Le coefficient est donc significatif.
Les variables sont de plus faiblement corrélées.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  0.04636187153020695  et sa p-value est de:  0.17436062065175273 .
Le coefficient n'est donc pas significatif.
RHO DE SPEARMAN:
Le coef de Spearman s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


On en conclut également à peu ou pas d'assocaition.

#### Inflation
Malgré le nombre d'occurences limité du terme, on regarde tout de même le lien entre l'apparition du terme inflation et le pourcentage d'inflation.

In [152]:
two_graphs(
    CPI_40['date'],
    abs(CPI_40['inf_an']),
    freq_sub['date'],
    freq_sub['inflation_rat'],
    'Inflation (%, val abs.)',
    "Article (%)", 
    "Date", 
    "Figure 14: Comparaison du pourcentage d'inflation annuelle (en valeur absolue) et du pourcentage d'apparition de inflation"
    )

In [154]:
corr_tests(abs(inf_freq['inf_an']),inf_freq['inflation_rat'])

TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  0.2483892512136041  et sa p-value est de:  5.559431938775148e-07 .
Le coefficient est donc significatif.
Les variables sont de plus faiblement corrélées.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  0.20015473167511955  et sa p-value est de:  3.261472038225439e-09 .
Le coefficient est donc significatif.
Les variables sont de plus faiblement corrélées.
RHO DE SPEARMAN:
Le coef de Spearman s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


In [156]:
inf_freq['inf_rat_ma'] = inf_freq['inflation_rat'].rolling(window=12).mean()
inf_freq_cleaned = inf_freq.replace([np.inf, -np.inf], np.nan).dropna(subset=['inf_ma', 'inf_rat_ma'])
corr_tests(abs(inf_freq_cleaned['inf_ma']),inf_freq_cleaned['inf_rat_ma'])

TEST DE CORRELATION DE PEARSON:
Le coefficient de corrélation de Pearson s'élève à:  0.4513566436225716  et sa p-value est de:  1.0116773308369948e-20 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.
TAU DE KENDALL:
Le tau de Kendall s'élève à:  0.3086861526252192  et sa p-value est de:  1.5139648225084495e-19 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.
RHO DE SPEARMAN:
Le coef de Spearman s'élève à:  0.37563007424985434  avec une p-value de:  1.0223850113248864e-14 .
Le coefficient est donc significatif.
Les variables sont de plus moyennement corrélées.


On peut en conclure à une corrélation moyenne entre la présence du terme inflation et le pourcentage d'inflation ce qui semble rassurant.

# 3- Modélisation

## 3.1- Régression linéaire

Une première façon de modéliser le lien entre inflation et traitement médiatique de cette thématique peut consister en une régression linéaire. Les perceptions d'inflation évaluées à travers le traitement médiatique de ce sujet peuvent constituer un indicateur d'inflation et être utilisées pour du *nowcasting* de l'inflation par exemple. On cherche donc alors à voir le pourcentage d'apparition des termes liés à l'inflation donne une prédiction satisfaisante du pourcentage d'inflation. Mais on peut également envisager une relation inverse. En effet, les perceptions d'inflation sont un objet d'étude à part entière en économie en raison de leurs multiples enjeux. On peut par exemple citer les nombreux travaux sur l'impact des annonces des banques centrales sur les anticipations d'inflation. Dès lors, si on prend les perceptions mesurées par le traitement médiatique comme variable d'intérêt, on peut chercher à évaluer à quel point l'inflation mesurée permet de les prédire. On s'abstiendra en revanche de toute analyse causale. 

In [171]:
def lin_reg(cov,dep):
    """
    Fonction réalisant une régression linéaire simple robuste à l'homoscédasticité.
    ===============================================================================
    Paramètres
    ===============================================================================
    cov: covariable
    dep: variable dépendante
    """
    x=cov 
    y=dep 

    x = sm.add_constant(x) #constante


    model = sm.RLM(y, x, M=sm.robust.norms.HuberT())
    results = model.fit()
    print(results.summary())

    # Calcul du R2
    # Valeurs prédites
    y_pred = results.predict(x)

    # Calculate R-squared
    rss = np.sum((y - y_pred) ** 2)  # Residual Sum of Squares
    tss = np.sum((y - np.mean(y)) ** 2)  # Total Sum of Squares
    r_squared = 1 - (rss / tss)

    print("R-squared:", r_squared)



### 3.1.1- Le traitement médiatique de l'inflation comme indicateur de l'inflation?

In [172]:
#Pourcentage d'inflation sur fréquences
lin_reg(inf_freq["total_rat"],abs(inf_freq["inf_an"]))

                    Robust linear Model Regression Results                    
Dep. Variable:                 inf_an   No. Observations:                  396
Model:                            RLM   Df Residuals:                      394
Method:                          IRLS   Df Model:                            1
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 27 Dec 2024                                         
Time:                        15:37:42                                         
No. Iterations:                    19                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -8.2117      1.129     -7.272      0.0

Le coefficient associé au pourcentage d'article mentionnant des termes liés à l'inflation est positif. Il s'élève de plus à 0.74 ce qui signifie qu'une augmentation de 1 point du pourcentage d'article est associée à une augmentation de 0.74 point du pourcentage d'inflation en valeur absolue. La p value est nulle donc ce coefficient est significatif. Le pourcentage d'article permet d'explique 17% de la variance de l'inflation. 

In [173]:
# En ma
lin_reg(inf_freq.loc[inf_freq["year"] > 1930, "tot_rat_ma"],abs(inf_freq.loc[inf_freq["year"] > 1930, "inf_ma"]))

                    Robust linear Model Regression Results                    
Dep. Variable:                 inf_ma   No. Observations:                  384
Model:                            RLM   Df Residuals:                      382
Method:                          IRLS   Df Model:                            1
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 27 Dec 2024                                         
Time:                        15:38:42                                         
No. Iterations:                    26                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.9331      1.260     -9.470      0.0

Le coefficient associé au pourcentage d'article mentionnant des termes liés à l'inflation en moyenne mobile est positif. Il s'élève de plus à 0.98 ce qui signifie qu'une augmentation de 1 point du pourcentage d'articlee est associé à une augmentation de 0.98 point du pourcentage d'inflation en moyenne mobile en valeur absolue. La p value est nulle donc ce coefficient est significatif. 

Le pourcentage d'articles permet en outre d'expliquer 25% de la variance d'inflation.

**Conclusion partielle**: Le pourcentage d'articles mentionnant des termes du champ lexical de l'inflation semble donc être une covariable utile pour prédire le pourcentage d'inflation. Il faut bien sûr envisager de la compléter par d'autres variables, ce qu'on pourrait faire en prolongement de notre sujet. On peut penser à d'autres mesures des perceptions, disponibles en temps réels si le but est le nowcasting, par exemple une analyse via les réseaux sociaux. Il faut aussi prendre en compte des facteurs plus causaux comme d'autres variables économiques, par exemple la croissance et le chômage.

### 3.1.2- L'inflation mesurée comme indicateur de l'inflation perçue dans les journaux

In [174]:
# Fréquences sur pourcentage d'Inflation
lin_reg(abs(inf_freq["inf_an"]),inf_freq["total_rat"])



                    Robust linear Model Regression Results                    
Dep. Variable:              total_rat   No. Observations:                  396
Model:                            RLM   Df Residuals:                      394
Method:                          IRLS   Df Model:                            1
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 27 Dec 2024                                         
Time:                        15:44:27                                         
No. Iterations:                    12                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         14.6469      0.122    120.485      0.0

Le coefficient associé au pourcentage d'inflation en valeur absolue est positif. Il s'élève de plus à 0.24 ce qui signifie qu'une augmentation de 1 point du pourcentage d'inflation est associée à une augmentation de 0.24 point du pourcentage d'articles mentionnant des termes liés à l'inflation. La p value est nulle donc ce coefficient est significatif. 

Le pourcentage d'inflation permet en outre d'expliquer 19% de la variance du pourcentage d'articles.

In [175]:
lin_reg(abs(inf_freq.loc[inf_freq["year"] > 1930, "inf_ma"]),inf_freq.loc[inf_freq["year"] > 1930, "tot_rat_ma"])

                    Robust linear Model Regression Results                    
Dep. Variable:             tot_rat_ma   No. Observations:                  384
Model:                            RLM   Df Residuals:                      382
Method:                          IRLS   Df Model:                            1
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 27 Dec 2024                                         
Time:                        15:47:05                                         
No. Iterations:                    16                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         14.6419      0.108    135.426      0.0

Le coefficient associé au pourcentage d'inflation en valeur absolue et en moyenne mobile est positif. Il s'élève de plus à 0.27 ce qui signifie qu'une augmentation de 1 point du pourcentage d'inflation est associée à une augmentation de 0.27 point du pourcentage d'articles en moyenne mobile mentionnant des termes liés à l'inflation. La p value est nulle donc ce coefficient est significatif. 

Le pourcentage d'inflation permet en outre d'expliquer 25% de la variance du pourcentage d'articles.

**Conclusion partielle**: le pourcentage d'inflation semble donc être une variable utile pour prédire les perceptions d'inflation. Il faut également envisager de la compléter par d'autres variables, même si les résultats seraient sûrement moins concluants que les précédents, dans la mesure on peut s'attendre à une prédictibilité moins forte des perceptions. Au-delà de chercher à prédire les perceptions d'inflation, on peut cependant déjà aller plus loin dans l'analyse en s'intéressant à la manière dont est perçue l'inflation grâce à un modèle d'analyse de sentiments.

## 3.2- Analyse de sentiments

In [8]:

import nltk
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rapam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rapam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Nettoyage du texte

In [10]:
#df=pd.read_parquet("ArticlesInflation/AllInflation.parquet")
with fs.open(f"{path}/AllInflation.parquet") as f:
    df = pd.read_parquet(f)

In [12]:
def clean_text(text):
    stop_words = set(stopwords.words("english"))
    text = text.lower()
    # Remove punctuation and stop words
    text = "".join([char for char in text if char not in string.punctuation])
    words = text.split()
    return " ".join([word for word in words if word not in stop_words])

def lemmatize_text(text):
    lemmatizer = nltk.WordNetLemmatizer()
    words = text.split()
    return " ".join([lemmatizer.lemmatize(word) for word in words])

def dereference_dico(dico):
    return dico["article"]

Ici la tokenization est implicite : les tokens sont chaque mot, cette approche peut être problématique si jamais les textes ont un format inhabituel comme des contractions ou des urls etc... Mais dans le cadre d'articles de presse, surtout d'anciens articles de presse il ne devrait y avoir aucun soucis

In [25]:
corpus=df["Article"]
corpus=corpus.apply(dereference_dico)
print(corpus.head())
corpus=corpus.apply(clean_text)
print(corpus.head())

0    BEST MAKES OF guaranteed tires at\n\n\nless th...
1    MAKE YOUR WALL PAPER clean and\n\n\nsweet agai...
2    STORIES. POEMS, PLAYS. ETC., are\n\n\nwanted f...
3    VV1ANTED-An experienced man on punch press and...
4    Have you lost a sum of money? Glasses. Pins an...
Name: Article, dtype: object
0    best makes guaranteed tires less dealers pay f...
1    make wall paper clean sweet simple formula suc...
2    stories poems plays etc wanted publication goo...
3    vv1antedan experienced man punch press eyelet ...
4    lost sum money glasses pins rings found surpri...
Name: Article, dtype: object


In [26]:
corpus=corpus.apply(lemmatize_text)
df["Treated"]=corpus

In [13]:
df["Article"]=df["Article"].apply(dereference_dico)
print(df.head())

                 Title     Date  \
0        The commoner.  1919-01   
1        The commoner.  1919-01   
2        The commoner.  1919-01   
3  New Britain herald.  1919-01   
4  New Britain herald.  1919-01   

                                             Article  
0  BEST MAKES OF guaranteed tires at\n\n\nless th...  
1  MAKE YOUR WALL PAPER clean and\n\n\nsweet agai...  
2  STORIES. POEMS, PLAYS. ETC., are\n\n\nwanted f...  
3  VV1ANTED-An experienced man on punch press and...  
4  Have you lost a sum of money? Glasses. Pins an...  


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

corpus=df["Treated"]
x = vectorizer.fit_transform(corpus)

In [16]:
print(type(x))
print(x.shape)

<class 'scipy.sparse._csr.csr_matrix'>
(3957020, 13988006)


In [33]:
toLabeled=df.sample(n=10000, random_state=42)
toLabeled=toLabeled[["Title","Date","Article"]]

In [34]:
print(toLabeled.head())

                        Title     Date  \
122716    New Britain herald.  1930-05   
557122      Norwich bulletin.  1920-07   
40954           Evening star.  1960-10   
73882   The Washington times.  1932-03   
131569          Evening star.  1952-10   

                                                  Article  
122716  New York, May ? [Pl-Curb prices huttered irreg...  
557122  number in town have received ti.e handsome. Da...  
40954   TRUSTEES SALE OF VALUABLE\nTWO-STORY BRICK DWE...  
73882   The market value of so repre-,\nsentative stoc...  
131569  the second floor there was displayed "a\ncoron...  


In [36]:
toLabeled.to_csv("ArticlesInflation/ToLabeled.csv")
toLabeledReduit=toLabeled.sample(n=100, random_state=42)
toLabeledReduit.to_csv("ArticlesInflation/ToLabeledReduit.csv")

Selection des articles parlant d'inflation à 100%

In [9]:
mots_inflation=["inflation", "disinflation", "inflationary", "deflation", "devaluation","recession","price level", "wage growth", "economic downturn", "monetary policy","inflation rate","interest rates", "price stability", "consumption basket", "purchasing power"]
articlesInflationSur=df[df["Article"].str.contains("|".join(mots_inflation), case=False)] # marche grâce aux regex
print(articlesInflationSur.head())

In [15]:
print(articlesInflationSur.shape)

(56337, 3)


In [20]:
toLabeled=articlesInflationSur.sample(n=100, random_state=42)
toLabeled.to_csv("ArticlesInflation/toLabelled.csv")

Utilisation de ChatGPT et vérifié à la main pour label les données

In [41]:
labeled=pd.read_csv("ArticlesInflation/ManuallyLabelledArticles.csv")
print(labeled.head())
print(labeled.shape)

   Unnamed: 0                     Title     Date  \
0       87446    Imperial Valley press.  1944-11   
1      133641  The Daily Alaska empire.  1935-09   
2       24042             Smyrna times.  1955-09   
3       37164             Evening star.  1934-06   
4       29819     The Washington times.  1934-05   

                                             Article     Label  
0  Here Gre 8 big reasons for buying tho\nsanst y...  Positive  
1  Designed to bring the farm program within the\...   Neutral  
2  A federal bearing to determine\nwhether the mi...   Neutral  
3  By the Associated Press.\n\n\nMembership in th...  Positive  
4  1--WE.\n\n\nThese three phases are not un-\nre...  Negative  
(100, 5)


In [42]:
print(labeled["Label"].value_counts())

Label
Neutral     38
Negative    35
Positive    27
Name: count, dtype: int64


In [43]:
texts=labeled["Article"]
labels=labeled["Label"]
texts=texts.apply(clean_text)
texts=texts.apply(lemmatize_text)

In [44]:
def numerical_label(label):
    if label=="Positive":
        return 0
    elif label=="Neutral":
        return 1
    else:
        return 2
labels=labels.apply(numerical_label)

In [46]:
print(labels.value_counts())
print(labels.isna().sum())

Label
1    38
2    35
0    27
Name: count, dtype: int64
0


In [47]:

vectorizer = TfidfVectorizer()
x=vectorizer.fit_transform(texts)

In [48]:
y=labels
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [49]:
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(x_train, y_train)

SVC()

In [53]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test set
y_pred = svm_model.predict(x_test)
# Evaluate the performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


[1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1]
83    1
53    2
70    2
45    1
44    2
39    1
22    1
80    1
10    0
0     0
18    2
30    1
73    2
33    0
90    1
4     2
76    0
77    2
12    2
31    0
Name: Label, dtype: int64
Accuracy: 0.4
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.37      1.00      0.54         7
           2       1.00      0.12      0.22         8

    accuracy                           0.40        20
   macro avg       0.46      0.38      0.25        20
weighted avg       0.53      0.40      0.28        20



c:\ProgramData\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
